In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from time import sleep
import csv
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import requests

print('- Finish importing packages')

- Finish importing packages


In [32]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(options=options)
# driver = webdriver.Chrome()
driver.get('https://vnexpress.net/')
sleep(1)
print('- Finish initializing a driver')

- Finish initializing a driver


In [33]:
page_source = BeautifulSoup(driver.page_source)
list_elements = page_source.find_all('h3', class_ = "title-news")
list_links = [e.find('a').get('href') for e in list_elements]

In [38]:
lst_dic = []

for link in list_links:
  
  driver.get(link)
  article_html = BeautifulSoup(driver.page_source)
  try:
    title = article_html.find_all('h1', class_ = 'title-detail')[0].get_text()
  except:
    title = "Unknown"

  try:
    date = article_html.find_all('span', class_ = 'date')[0].get_text()
  except:
    date = ""

  author_list = article_html.find_all('p', class_ = 'author_mail')
  
  try:
    if len(author_list) != 0:
      author = author_list[0].get_text()
    else:
      author = article_html.find_all('h3', class_ = 'title-news')[0].get_text()
  except:
    author = "Unknown"

  try:
    lst_content = [e.get_text() for e in article_html.find('article', class_ = 'fck_detail').findAll('p', class_ = "Normal")]
    content = "\n".join(lst_content)
  except:
    content = ""

  try:
    list_img_links = [e.get('src') for e in article_html.find('article', class_ = 'fck_detail').findAll('img') if e.get('src').startswith("https")]
  except:
    list_img_links = []

  dic = {
    "title": title,
    "date": date,
    "author": author,
    "content": content,
    "imgs": list_img_links
  }

  lst_dic.append(dic)

In [43]:
len(lst_dic)

60

In [49]:
# import urllib
# from pymongo import MongoClient
# client = MongoClient("mongodb+srv://vietthangc1:"+urllib.parse.quote_plus('f2bdx@*-uLAZz!f')+"@cluster0.le7ea.mongodb.net/test")
# db = client.vnexpress

import json
f = open("../app/data/article.json", 'w', encoding='utf-8')
json.dump(lst_dic, f)
f.close()

In [4]:
def get_link(driver=driver):
    page_source = BeautifulSoup(driver.page_source)
    content = page_source.find_all('div', class_="ListAds___1z6Pv")[1]
    link_ads = content.find_all('a')
    for i in range(len(link_ads)):
        if i%2 == 0:
            link = 'https://nha.chotot.com' + link_ads[i].get('href')
            lst_link_tong.append(link)

def get_info(keyword, col_name):
    var = '-'
    for i in range(len(lst_params)):
        try:
            if keyword in lst_params[i].find('div', class_ = "media-body media-middle").find('span').find_all('span')[0].get_text().lower():
                var = lst_params[i].find('div', class_ = "media-body media-middle").find('span').find_all('span')[1].get_text()
                break
        except AttributeError:
            continue  
    dic[col_name].append(var)

In [10]:
lst_link_tong = []
for url in lst_url:
    print(url)
    driver.get(url)
    page_source = BeautifulSoup(driver.page_source)
    content = page_source.find_all('div', class_="ListAds_ListAds__1z6Pv")[0]
    # print(content)
    link_ads = content.find_all('a')
    for i in range(len(link_ads)):
        if i%2 == 0:
            link = 'https://nha.chotot.com' + link_ads[i].get('href')
            lst_link_tong.append(link)

https://nha.chotot.com/dong-nai/thanh-pho-bien-hoa/phuong-quyet-thang/mua-ban-nha-dat?page=2


In [11]:
content

<div class="ListAds_ListAds__1z6Pv col-xs-12 no-padding"><ul><div role="button" tabindex="0"><li class="AdItem_wrapperAdItem__1hEwM AdItem_big__2Sqod"><a class="AdItem_adItem__2O28x" href="/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/95951468.htm"><div class="Layout_left__F_Zl-"><div class="AdThumbnail_AdThumbnail__3AaSd AdThumbnail_big__1_Hy1 false"><picture class="webpimg-container"><img alt="Bán nhà,90m2,SHR, Thổ cư Chỉ 4,75 tỷ P Quyết Thắng" class="AdThumbnail_thumbnail__HD_VW priority" height="128" src="https://cdn.chotot.com/4dZwHUQpgyOxMRbERECmDNh_UlaXn1sAassxdtPTeao/preset:listing/plain/f1e87f5db8d6527f49d848ae24cb4d85-2776921687600037535.jpg" title="Bán nhà,90m2,SHR, Thổ cư Chỉ 4,75 tỷ P Quyết Thắng" width="128"/></picture><div class="AdThumbnail_imageGroup__25Vc-"><div class="AdThumbnail_imageNum__1n0ef">5</div></div></div></div><div class="Layout_right__2f2Ss"><div class="AdBody_AdBody__mf6Ri"><h3 class="commonStyle_adTitle__1VYG8">Bán nhà,90m2,SHR, Thổ cư Chỉ 4,75 tỷ P Quyế

In [6]:
dic = {
    'maso': [],
    'price': [],
    'price_m2': [],
    'area_dat': [],
    'ngang': [],
    'dai': [],
    'area_nha': [],
    'room': [],
    'desc': [], 
    'st': [],
    'ward': []
}

for ad in lst_link_tong[:176]:
    maso = ad.split('/')[-1].split('.')[0]
    dic['maso'].append(maso)

    driver.get(ad)
    page_source = BeautifulSoup(driver.page_source)

    address1, address2 = '-', '-'
    try:
        address_area = page_source.find_all('div', class_ = 'col-xs-12 no-padding')[0]
        address1 = address_area.get_text().split('Phường')[0][:-2]
        address2 = address_area.get_text().split('Phường')[1].split(',')[0]
    except:
        pass
    dic['st'].append(address1)
    dic['ward'].append(address2)

    area_price = ''
    try:
        area_price = page_source.find('span', class_ = 'price___1Ps8j').get_text().split('-')[0]
    except AttributeError:
        pass
    dic['price'].append(area_price)

    desc = page_source.find('p', class_ = 'AdDecription_adBody__1c8SG').get_text()
    dic['desc'].append(desc)

    params = page_source.find_all('div', class_ = 'col-xs-12 no-padding')[1]
    lst_params = params.find_all('div', class_ = 'col-md-6 no-padding AdParam_adParamItem__1o-dd')

    get_info('diện tích đất', 'area_dat')
    get_info('giá', 'price_m2')
    get_info('số phòng ngủ', 'room')
    get_info('ngang', 'ngang')
    get_info('dài', 'dai')
    get_info('diện tích sử dụng', 'area_nha')

    
    

In [7]:
df = pd.DataFrame(dic)
df

,maso,price,price_m2,area_dat,ngang,dai,area_nha,room,desc,st,ward
0,95951468,,"52,78 triệu/m²",90 m²,5 m,18 m,90 m²,4 phòng,"💸💸 Bán nhà 1T 2L , 90m2 ,SHR, Thổ cư , đã Hoàn...",Đường Hà Huy Giáp,Quyết Thắng
1,95378022,,"27,66 triệu/m²",94 m²,4 m,-,94 m²,2 phòng,"🔥🔥Bán nhà trung tâm Biên Hòa - Quyết Thắng, gầ...",Võ Thị Sáu,Quyết Thắng
2,95829019,,"37,74 triệu/m²",53 m²,3 m,16 m,85 m²,3 phòng,Chủ gửi bán nhà 2 mặt hẻm P.Quyết Thắng \n👉🏻Nh...,"68/79, Đường Hà Huy Giáp",Quyết Thắng
3,94955295,,"138,89 triệu/m²",72 m²,4 m,18 m,216 m²,4 phòng,☘️ MỘT CĂN MẶT TIỀN VTSAU . 1 TRỆT 2 LẦU .\n- ...,Đường Võ Thị Sáu,Quyết Thắng
4,95608637,,"15,19 triệu/m²",167.9 m²,4.5 m,25 m,112.5 m²,2 phòng,"🌳 BÁN NHÀ 112,5m2 Phường Quyết Thắng 🌳\n\n💁 Di...",Đường Võ Thị Sáu,Quyết Thắng
5,95767073,,"39,62 triệu/m²",53 m²,-,-,-,3 phòng,Chủ gửi bán nhà 2 mặt hẻm P.Quyết Thắng \n👉🏻Nh...,Hà Huy Giáp,Quyết Thắng
6,94817838,,39 triệu/m²,100 m²,4 m,25 m,100 m²,2 phòng,"Nhà phường Quyết Thắng, ngay sau sở ngoại vụ t...",Đường Võ Thị Sáu,Quyết Thắng
7,95729557,,"1,46 triệu/m²",41 m²,3.5 m,11.7 m,80 m²,2 phòng,Gia đình cần bán căn nhà định cư hơn 40 năm.\n...,Đường Hà Huy Giáp,Quyết Thắng
8,95220690,,"32,14 triệu/m²",42 m²,-,-,-,2 phòng,Bán căn nhà phường Quyết Thắng\n✅ Diện Tích:...,Đường Hà Huy Giáp,Quyết Thắng
9,94725535,,39 triệu/m²,100 m²,4 m,25 m,-,2 phòng,"Nhà phường Quyết Thắng, ngay sau sở ngoại vụ t...",Đường Võ Thị Sáu,Quyết Thắng


In [8]:
df.to_csv('bienhoa.csv', encoding = 'utf-8')

In [9]:
lst_link_tong

['https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/95951468.htm',
 'https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/95378022.htm',
 'https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/95829019.htm',
 'https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/94955295.htm',
 'https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/95608637.htm',
 'https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/95767073.htm',
 'https://nha.chotot.com/mua-ban-nha-dat-thanh-pho-bien-hoa-dong-nai/94817838.htm',
 'https://nha.chotot.com/mua-ban-thanh-pho-bien-hoa-dong-nai/95729557.htm',
 'https://nha.chotot.com/mua-ban-thanh-pho-bien-hoa-dong-nai/95220690.htm',
 'https://nha.chotot.com/mua-ban-thanh-pho-bien-hoa-dong-nai/94725535.htm',
 'https://nha.chotot.com/mua-ban-thanh-pho-bien-hoa-dong-nai/95553136.htm',
 'https://nha.chotot.com/mua-ban-thanh-pho-bien-hoa-dong-nai/95547085.htm',
 'https://nha.chotot.com/mua-ban